#PROJECT 1

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import re
from pyspark.sql.window import Window
from pyspark import SparkContext
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("my project 1").getOrCreate()
sc = spark.sparkContext

# Read a CSV into a dataframe
# There is a smarter version, that will first check if there is a Parquet file and use it
def load_PD_file(filename_or_dir, schema) :
    dataPath = "/mnt/ddscoursedatastorage/fwm-stb-data/" + filename_or_dir
    df = spark.read.format("csv")\
      .option("header","false")\
      .option("delimiter", "|")\
      .schema(schema)\
      .load(dataPath)
    return df

df1 = spark.read.csv("/mnt/ddscoursedatastorage/dds-students/test.csv")


# Reading the Reference Parquet files

ref_data = spark.read.parquet('/ref_data_raw').withColumnRenamed("_device-id","device_id")\
                                                .withColumnRenamed("_dma","dma")\
                                                .withColumnRenamed("_dma-code","dma_code")\
                                                .withColumnRenamed("_household-id","household_id")\
                                                .withColumnRenamed("_household-type","household_type")\
                                                .withColumnRenamed("_system-type","system_type")\
                                                .withColumnRenamed("_zipcode","zipcode")
ref_data_count = ref_data.count()
print(ref_data_count)
#ref_data.limit(5).show()


# Reading the Daily Programs CSV file

daily_prog_schema =  StructType([StructField('prog_code',StringType()),
                     StructField('title',StringType()),
                     StructField('genre',StringType()),
                     StructField('air_date',StringType()),
                     StructField('air_time',StringType()),
                     StructField('Duration',FloatType())
                                       ])
daily_prog_data = load_PD_file("Daily program data/" , daily_prog_schema  )

#daily_prog_data.limit(3).show()

# Reading the 2.5% sample of the viewing data from a Parquet file

viewing_data = spark.read.parquet('/sample_viewing_2_5percent')

print(f'There are {viewing_data.count():,} entries in viewing_data dataframe!')
#viewing_data.show(5)

# Reading the Demographic CSV file

demographic_schema =  StructType([StructField('household_id',StringType()),
                      StructField('household_size',IntegerType()),
                      StructField('num_adults',IntegerType()),
                      StructField('num_generations',IntegerType()),
                      StructField('adult_range',StringType()),
                      StructField('marital_status',StringType()),
                      StructField('race_code',StringType()),
                      StructField('presence_children',StringType()),
                      StructField('num_children',IntegerType()),
                      StructField('age_children',StringType()), #format like range - 'bitwise'
                      StructField('age_range_children',StringType()),
                      StructField('dwelling_type',StringType()),
                      StructField('home_owner_status',StringType()),
                      StructField('length_residence',IntegerType()),
                      StructField('home_market_value',StringType()),
                      StructField('num_vehicles',IntegerType()),
                      StructField('vehicle_make',StringType()),
                      StructField('vehicle_model',StringType()),
                      StructField('vehicle_year',IntegerType()),
                      StructField('net_worth',IntegerType()),
                      StructField('income',StringType()),
                      StructField('gender_individual',StringType()),
                      StructField('age_individual',IntegerType()),
                      StructField('education_highest',StringType()),
                      StructField('occupation_highest',StringType()),
                      StructField('education_1',StringType()),
                      StructField('occupation_1',StringType()),
                      StructField('age_2',IntegerType()),
                      StructField('education_2',StringType()),
                      StructField('occupation_2',StringType()),
                      StructField('age_3',IntegerType()),
                      StructField('education_3',StringType()),
                      StructField('occupation_3',StringType()),
                      StructField('age_4',IntegerType()),
                      StructField('education_4',StringType()),
                      StructField('occupation_4',StringType()),
                      StructField('age_5',IntegerType()),
                      StructField('education_5',StringType()),
                      StructField('occupation_5',StringType()),
                      StructField('polit_party_regist',StringType()),
                      StructField('polit_party_input',StringType()),
                      StructField('household_clusters',StringType()),
                      StructField('insurance_groups',StringType()),
                      StructField('financial_groups',StringType()),
                      StructField('green_living',StringType())
                                       ])

demographic_data = load_PD_file("demographic/" , demographic_schema  )  

#display(demographic_data.limit(1))


203581233
There are 130,289,194 entries in viewing_data dataframe!


#PART 2

In [0]:
ref_data = ref_data.select(['device_id','dma','household_id']).na.drop().dropDuplicates()
ref_data.cache()
daily_prog_data.cache()

#QUESTION 2.1
prog_data_sep_genre = daily_prog_data.select("prog_code","genre")\
    .withColumn("genre",explode(split("genre",",")))\
        .na.drop().dropDuplicates()

ref_dma_data = ref_data.select("device_id","dma").na.drop().dropDuplicates()

#right join to not remove the dmas without viewings   
extended_view_data = viewing_data.join(broadcast(prog_data_sep_genre),on="prog_code").join(ref_dma_data,on="device_id",how="right")

total_views_dma_genre = extended_view_data.groupBy("dma","genre")\
    .agg(countDistinct("prog_code","event_date","event_time").alias("total_views"))\
        .orderBy("dma",desc("total_views")).repartition("dma") #order by total views desc

result_2_1 = total_views_dma_genre.groupBy("dma").agg(collect_list("genre").alias("genres")) 
display(result_2_1)
print("QUESTION 2.1")
top_5 = result_2_1.where((col("dma")=="Waco-Temple-Bryan") | (col("dma")=="New York")|(col("dma")=="Washington, DC (Hagrstwn)"))
rows = top_5.collect()
for row in rows:
    print(f"DMA: {row.dma} , TOP 5 GENRES: {row.genres[:5]}")
print("")

dma,genres
Sherman-Ada,"List(Reality, News, Sitcom, Children, Talk, House/garden, Comedy, Animated, Adventure, Consumer, Documentary, Crime, Drama, Action, Crime drama, Fantasy, Entertainment, Home improvement, Newsmagazine, Music, Mystery, Sports event, Educational, Football, Special, Game show, Shopping, Travel, Suspense, Soap, Animals, Sports non-event, Religious, History, Law, Science, Science fiction, Nature, Romance, Auto, Interview, Docudrama, How-to, Outdoors, Medical, Romance-comedy, Bus./financial, Politics, Horror, Public affairs, Sports talk, Cooking, Fashion, Western, Paranormal, Health, Variety, Comedy-drama, Auto racing, Playoff sports, Biography, Musical, Weather, Basketball, Golf, Technology, Dance, Anthology, Musical comedy, Fishing, War, Hunting, Historical drama, Baseball, Parenting, Soccer, Art, Community, Pro wrestling, Collectibles, Spanish, Gay/lesbian, Martial arts, Awards, Self improvement, Exercise, Anime, Auction, Action sports, Hockey, Debate, Military, Boat, Boxing, Environment, Aviation, Event, Volleyball, Horse, Figure skating, Standup, Rugby, Rodeo, Tennis, Arts/crafts, Gaming, Shooting, Softball, Mixed martial arts, Poker, Parade, Snowboarding, Equestrian, Dog show, Motorcycle racing, Gymnastics, Skiing, Card games, Motorsports, Miniseries, Adults only, Kayaking, Track/field, Sailing, Swimming, Fundraiser, Pool, Beach volleyball, Running, Beach soccer, Agriculture, Bicycle racing, Skateboarding)"
Wheeling-Steubenville,"List(Reality, News, Sports event, Documentary, Talk, Adventure, Drama, Sports non-event, Crime drama, Action, Sitcom, Comedy, Outdoors, Mystery, Children, Animated, Suspense, Football, Crime, Shopping, Baseball, Game show, Special, Fantasy, Consumer, Sports talk, Western, Music, Basketball, House/garden, Entertainment, Nature, Cooking, Newsmagazine, Law, Playoff sports, Animals, Weather, Science fiction, History, Educational, Romance, Medical, Politics, Travel, Hunting, Public affairs, Interview, Golf, Home improvement, Horror, Science, Variety, Religious, How-to, Romance-comedy, Bus./financial, Auto, Paranormal, Auto racing, Docudrama, War, Comedy-drama, Health, Fishing, Fashion, Soap, Biography, Historical drama, Collectibles, Community, Auction, Hockey, Anthology, Musical, Dance, Soccer, Mixed martial arts, Boxing, Parenting, Musical comedy, Aviation, Art, Martial arts, Pro wrestling, Shooting, Technology, Military, Adults only, Poker, Card games, Softball, Tennis, Archery, Action sports, Agriculture, Awards, Horse, Environment, Event, Self improvement, Track/field, Debate, Anime, Drag racing, Miniseries, Figure skating, Rodeo, Bicycle racing, Equestrian, Exercise, Motorsports, Volleyball, Parade, Motorcycle racing, Gaming, Standup, Computers, Gymnastics, Motorcycle, Dog show, Beach volleyball, Wrestling, Skiing, Gay/lesbian, Bowling, Rugby, Boat, Beach soccer, Diving, Arm wrestling, Swimming, Arts/crafts, Fundraiser, Spanish, Snowboarding, Billiards, Running, Lacrosse, Snowmobile, Cheerleading, Olympics, Pool, Kayaking, Theater, Triathlon, Skateboarding, Water polo, Watersports, Weightlifting, Music special, Luge, Cricket, Performing arts, Holiday, Yacht racing, Sailing, Bobsled, Intl soccer, Field hockey, Curling, Darts, Holiday-children special, Holiday special, Boat racing)"
Little Rock-Pine Bluff,"List(Reality, News, Talk, Drama, Documentary, Sitcom, Sports event, Comedy, Adventure, Children, Action, Animated, Sports non-event, Crime drama, Suspense, Football, Fantasy, Crime, Mystery, House/garden, Special, Music, Science fiction, Entertainment, Sports talk, Consumer, Western, Cooking, Game show, Horror, Basketball, Law, Politics, Interview, Newsmagazine, Educational, Outdoors, History, Playoff sports, Shopping, Weather, Home improvement, Public affairs, Religious, Travel, Baseball, Romance, Medical, Animals, Paranormal, Science, How-to, Golf, Bus./financial, Auto, Romance-comedy, Nature, War, Comedy-drama, Health, Auto racing, Hunting, Docudrama, Fashion, Biography, His

QUESTION 2.1
DMA: Waco-Temple-Bryan , TOP 5 GENRES: ['Reality', 'News', 'Sitcom', 'Talk', 'Sports event']
DMA: Washington, DC (Hagrstwn) , TOP 5 GENRES: ['Reality', 'News', 'Sitcom', 'Comedy', 'Drama']
DMA: New York , TOP 5 GENRES: ['News', 'Reality', 'Talk', 'Sitcom', 'Documentary']



In [0]:
#Part 2.2
#NUMERICAL COLUMNS FIX
demographic_wealth_data = demographic_data.select("household_id", "net_worth", "income").distinct().\
    withColumn("income",when(col('income')=='A',11).otherwise(when(col('income')=='B',11)\
        .otherwise(when(col('income')=='C',12).otherwise(when(col("income")=='D',13)\
            .otherwise(when(col("income").isNull(),0).otherwise(col("income")))))))\
                .withColumn("income",col("income").cast('int'))\
                    .withColumn("net_worth",when(col('net_worth')=='A',11).otherwise(when(col('net_worth')=='B',11)\
                        .otherwise(when(col('net_worth')=='C',12).otherwise(when(col("net_worth")=='D',13)\
                            .otherwise(when(col("net_worth").isNull(),0).otherwise(col("net_worth")))))))\
                                .withColumn("net_worth",col("net_worth").cast('int'))

#GET MAX NETWORTH AND INCOME   
max_networth = demographic_wealth_data.agg(max("net_worth")).collect()[0][0]
max_income = demographic_wealth_data.agg(max("income")).collect()[0][0]

dma_wealth_data = ref_data.select("dma","household_id").distinct()\
    .join(broadcast(demographic_wealth_data),on="household_id").repartition("dma").groupBy("dma")\
        .agg(avg('net_worth').alias('avg_net_worth'), avg('income').alias('avg_income'))

dma_normalized_avg = dma_wealth_data.withColumn("WEALTH SCORE",( (col('avg_net_worth')/max_networth) + (col('avg_income')/max_income))).withColumnRenamed("dma","DMA").select("DMA","WEALTH SCORE").orderBy(desc("WEALTH SCORE"))


all_genres = set([row.genre for row in prog_data_sep_genre.select('genre').collect()])
counter = 1
schema = StructType([
    StructField('DMA', StringType(), nullable=False),
    StructField('WEALTH SCORE', DoubleType(), nullable=False),
    StructField('ORDERED LIST OF GENRES', ArrayType(StringType()), nullable=False)
])
result_2_2 = spark.createDataFrame([],schema)
dict_3={} #key:dma name value:unique top 8 list - this dictionary is used for question 3
print("QUESTION 2.2")
for row in dma_normalized_avg.collect():
    dma_name = row["DMA"]
    dma_wealth = row["WEALTH SCORE"]
    dma_top_gen = result_2_1.filter(col('dma')==dma_name).collect()
    if(len(dma_top_gen)!=0):
        current_list = dma_top_gen[0][1]
        top_8 = [genre for genre in current_list if genre in all_genres][:8]
        all_genres = all_genres-set(top_8)
        
    else:
        top_8 = []
    dict_3[dma_name]=top_8
    new_row = spark.createDataFrame([(dma_name,dma_wealth,top_8)],schema)
    result_2_2 = result_2_2.union(new_row)
result_2_2.show(25,truncate=False)
display(result_2_2)   

QUESTION 2.2
+--------------------------+------------------+-----------------------------------------------------------------------------------------------------------+
|DMA                       |WEALTH SCORE      |ORDERED LIST OF GENRES                                                                                     |
+--------------------------+------------------+-----------------------------------------------------------------------------------------------------------+
|San Antonio               |1.623931623931624 |[]                                                                                                         |
|Baltimore                 |1.3367165242165242|[News, Reality, Talk, Comedy, Sports event, Sitcom, Drama, Documentary]                                    |
|San Francisco-Oak-San Jose|1.330195301209794 |[Music, Children, Animated, Adventure, Educational, Fantasy, Sports non-event, Crime]                      |
|Detroit                   |1.2932170709948487|[Act

DMA,WEALTH SCORE,ORDERED LIST OF GENRES
San Antonio,1.623931623931624,List()
Baltimore,1.3367165242165242,"List(News, Reality, Talk, Comedy, Sports event, Sitcom, Drama, Documentary)"
San Francisco-Oak-San Jose,1.330195301209794,"List(Music, Children, Animated, Adventure, Educational, Fantasy, Sports non-event, Crime)"
Detroit,1.2932170709948487,"List(Action, Crime drama, Suspense, House/garden, Entertainment, Consumer, Sports talk, Football)"
Austin,1.254676045163184,"List(Special, Mystery, Cooking, Politics, Newsmagazine, Interview, Game show, Western)"
Sacramnto-Stkton-Modesto,1.2187876275636091,"List(Shopping, Law, Science fiction, Baseball, Travel, Public affairs, Horror, Basketball)"
Seattle-Tacoma,1.2176017291607932,"List(Home improvement, Outdoors, History, Religious, Medical, Romance, How-to, Playoff sports)"
Philadelphia,1.199571059220182,"List(Weather, Animals, Nature, Romance-comedy, Science, Variety, Auto racing, War)"
Harrisburg-Lncstr-Leb-York,1.1981715763228369,"List(Bus./financial, Golf, Hockey, Auto, Paranormal, Comedy-drama, Health, Tennis)"
Cleveland-Akron (Canton),1.1952732452732453,"List(Soap, Hunting, Fashion, Collectibles, Anthology, Docudrama, Mixed martial arts, Fishing)"


#PART 3

In [0]:
#QUESTION 3
#create new directory with the dma name
def createNewDirectory(new_dir_name):
    result = 'Question3/' + new_dir_name
    return result
    
#the schema for the new df
daily_prog_schema =  StructType([StructField('prog_code',StringType()),
                     StructField('title',StringType()),
                     StructField('genre',StringType()),
                     StructField('air_date',StringType()),
                     StructField('air_time',StringType()),
                     StructField('Duration',FloatType()),
                     StructField('genre_asso',StringType()) #genre_asso is the genre header of the file(each genre get a file, so that specify which file genre is)
                                       ])
#iterate the dma's and their top 8
for dma,genres in dict_3.items():
    output_dir = createNewDirectory(dma)
    new_df = spark.createDataFrame([],daily_prog_schema)
    if len(genres)!=0:
        for c_genre in genres:
            from_daily = daily_prog_data.filter(array_contains(split(col('genre'), ","), c_genre))
            from_daily = from_daily.withColumn('genre_asso', lit(c_genre))
            new_df = new_df.union(from_daily).na.drop().dropDuplicates()
            new_df.write.csv(output_dir + "/" + c_genre, header=True, mode="overwrite") #create a csv file with the genre df

In [0]:
#END OF PROJECT 1